# Getting my labelled data

#### Script to find:

1) all hospital admissions in which the patient was invasively ventilated in the ICU 

2) in which of these admissions were patients placed back on the ventilator within 48 hours

3) remove patients that died while on the ventilator

4) look for > 2 'invasive ventilation' events in any particular hospital admission

#### import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#### read data from the file 'proecdureevents.csv' which contains lists all invasive ventilation procedures occuring in patients in the icu

In [2]:
df = pd.read_csv('procedureevents.csv')
#df.info()

#### find all 'invasive ventilation' events (identified by itemid code: 225792)

In [3]:
vdf= df[df['itemid']==225792]
vdf['hadm_id'].nunique()

26409

In [4]:
vdf['hadm_id'].count()-vdf['hadm_id'].nunique()

4223

We can see from the above table that there were 30632 ventilation events in the study, from 26409 different hospital admissions. This means that in 4223 of these admissions, the patient was ventilated multiple times.

#### Get list of admissions where the patients was only ventilated once

In [5]:
num_ventilations = vdf['hadm_id'].value_counts()
svdf = vdf[vdf['hadm_id'].isin(num_ventilations.index[num_ventilations == 1])]

In [16]:
svdf = df.assign(re_intub_class=0)
svdf

,subject_id,hadm_id,stay_id,starttime,endtime,storetime,itemid,value,valueuom,location,...,totalamount,totalamountuom,isopenbag,continueinnextdept,cancelreason,statusdescription,comments_date,originalamount,originalrate,re_intub_class
0,15693895,21203492,34766377,2147-06-05 11:25:00,2147-06-05 11:26:00,2147-06-06 11:25:00,225399,1.000000,None,NaN,...,NaN,NaN,0,0,0,FinishedRunning,NaN,1.0,0,0
1,15693895,21203492,34766377,2147-06-05 22:00:00,2147-06-05 22:01:00,2147-06-06 11:25:00,224385,1.000000,None,NaN,...,NaN,NaN,0,0,0,FinishedRunning,NaN,1.0,0,0
2,12481680,26876606,30988462,2141-01-23 12:00:00,2141-01-23 18:33:00,2141-01-23 18:33:51.14,224275,393.000000,min,NaN,...,NaN,NaN,1,0,0,FinishedRunning,NaN,393.0,1,0
3,12481680,26876606,30988462,2141-01-23 12:00:00,2141-01-23 18:33:00,2141-01-23 18:33:51.14,224275,393.000000,min,NaN,...,NaN,NaN,1,0,0,FinishedRunning,NaN,393.0,1,0
4,15614172,27424463,31265727,2153-07-29 18:13:00,2153-07-31 01:25:00,2153-07-31 01:26:00,224277,1872.000000,min,RL Post Forearm,...,NaN,NaN,1,1,0,FinishedRunning,NaN,1872.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
689841,19119144,28226348,38018422,2188-11-03 21:45:00,2188-11-03 21:46:00,2188-11-03 23:57:00,228129,1.000000,None,NaN,...,NaN,NaN,0,0,0,FinishedRunning,NaN,1.0,0,0
689842,13670990,24273866,37934708,2141-12-19 13:03:00,2141-12-19 22:54:00,2141-12-19 17:53:00,224263,0.410417,day,Right Femoral.,...,NaN,NaN,0,0,0,Stopped,NaN,1.0,1,0
689843,13670990,24273866,37934708,2141-12-19 13:06:00,2141-12-19 22:55:00,2141-12-19 22:55:15.38,224277,589.000000,min,NaN,...,NaN,NaN,1,0,0,FinishedRunning,NaN,589.0,1,0
689844,13670990,24273866,37934708,2141-12-19 13:22:00,2141-12-19 22:55:00,2141-12-19 22:55:15.38,229351,573.000000,min,NaN,...,NaN,NaN,1,0,0,FinishedRunning,NaN,573.0,1,0


#### Get list of admissions where the patients was ventilated a second time within 48 hours

In [17]:
mvdf = vdf[vdf['hadm_id'].isin(num_ventilations.index[num_ventilations == 2])]
sorted_mvdf = mvdf.sort_values(['hadm_id', 'starttime'], ascending=[True, True])  # sort table by hadm_id followed by starttime

sorted_mvdf['starttime'] = pd.to_datetime(sorted_mvdf['starttime']) # convert times to datetime objects
sorted_mvdf['endtime'] = pd.to_datetime(sorted_mvdf['endtime']) 

first_vent = sorted_mvdf.iloc[::2] # slice dataframe so only have first ventilation events
second_vent=sorted_mvdf.iloc[1::2] # slice dataframe so only have second ventilation events

vent_diff = pd.merge(left = first_vent,right = second_vent,on = 'hadm_id')  # merge table so start/endtimes for both ventilaton events from an admission are in the same row for 
vent_diff['timetosecondvent']= vent_diff['starttime_y'] - vent_diff['endtime_x'] # add column of time difference between end of first ventilation event and beginning of second ventilation event 
vent_diff['timetosecondvent'] = vent_diff['timetosecondvent'] / np.timedelta64(1, 'h') # convert time to hours
re_vent = vent_diff[vent_diff['timetosecondvent']<48]

In [18]:
re_vent.columns = ['subject_id_x', 'hadm_id', 'stay_id_x', 'starttime_x', 'endtime',
       'storetime_x', 'itemid_x', 'value', 'valueuom', 'location_x',
       'locationcategory_x', 'orderid_x', 'linkorderid_x',
       'ordercategoryname_x', 'secondaryordercategoryname_x',
       'ordercategorydescription_x', 'patientweight_x', 'totalamount_x',
       'totalamountuom_x', 'isopenbag_x', 'continueinnextdept_x',
       'cancelreason_x', 'statusdescription_x', 'comments_date_x',
       'originalamount_x', 'originalrate_x', 'subject_id_y', 'stay_id_y',
       'starttime_y', 'endtime_y', 'storetime_y', 'itemid_y', 'value_y',
       'valueuom_y', 'location_y', 'locationcategory_y', 'orderid_y',
       'linkorderid_y', 'ordercategoryname_y', 'secondaryordercategoryname_y',
       'ordercategorydescription_y', 'patientweight_y', 'totalamount_y',
       'totalamountuom_y', 'isopenbag_y', 'continueinnextdept_y',
       'cancelreason_y', 'statusdescription_y', 'comments_date_y',
       'originalamount_y', 'originalrate_y', 'timetosecondvent']

In [19]:
re_vent = re_vent.assign(re_intub_class=1)
re_vent

,subject_id_x,hadm_id,stay_id_x,starttime_x,endtime,storetime_x,itemid_x,value,valueuom,location_x,...,totalamountuom_y,isopenbag_y,continueinnextdept_y,cancelreason_y,statusdescription_y,comments_date_y,originalamount_y,originalrate_y,timetosecondvent,re_intub_class
2,17223646,20023461,36478017,2152-11-03 11:10:00,2152-11-03 16:44:00,2152-11-03 16:45:00,225792,334.0,min,NaN,...,NaN,1,0,0,FinishedRunning,NaN,6831.0,1,0.766667,1
3,16779215,20024229,38542512,2132-08-27 16:00:00,2132-08-28 17:30:00,2132-08-28 17:43:00,225792,1530.0,min,NaN,...,NaN,1,0,0,FinishedRunning,NaN,5554.0,1,34.716667,1
4,13605251,20025172,38493547,2139-12-31 14:02:00,2140-01-04 11:09:00,2140-01-04 11:09:00,225792,5587.0,min,NaN,...,NaN,1,0,0,FinishedRunning,NaN,23548.0,1,21.133333,1
5,16976998,20034762,30778603,2164-06-07 08:10:00,2164-06-08 14:00:00,2164-06-08 17:51:00,225792,1790.0,min,NaN,...,NaN,1,0,0,FinishedRunning,NaN,1901.0,1,45.183333,1
6,15878712,20035700,30421802,2123-02-01 10:22:00,2123-02-05 14:52:00,2123-02-05 18:01:00,225792,6030.0,min,NaN,...,NaN,1,0,0,FinishedRunning,NaN,2014.0,1,34.633333,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2337,16691924,29960248,32296652,2146-12-06 12:55:00,2146-12-09 14:23:00,2146-12-09 14:25:00,225792,4408.0,min,NaN,...,NaN,1,0,0,FinishedRunning,NaN,3136.0,1,10.283333,1
2338,16796371,29962016,38636381,2135-10-21 09:40:00,2135-10-30 11:45:00,2135-10-30 12:27:00,225792,13085.0,min,NaN,...,NaN,1,0,0,FinishedRunning,NaN,2842.0,1,0.583333,1
2341,10020944,29974575,33972149,2131-02-27 17:00:00,2131-03-03 19:54:00,2131-03-04 09:55:00,225792,5934.0,min,NaN,...,NaN,1,0,0,FinishedRunning,NaN,5446.0,1,0.283333,1
2343,19652570,29987115,35887309,2148-02-17 14:00:00,2148-02-19 10:00:00,2148-02-19 10:02:00,225792,2640.0,min,NaN,...,NaN,1,0,0,FinishedRunning,NaN,395.0,1,26.350000,1


In [20]:
sample_vents = pd.concat([svdf[['hadm_id','endtime','value','valueuom']],re_vent[['hadm_id','endtime','value','valueuom']]])

In [21]:
sample_vents.to_csv('sample_vents.csv',index= False)

In [44]:
svents = pd.read_csv('sample_vents.csv')

In [45]:
svents

,hadm_id,endtime,value,valueuom
0,25015072,2152-04-10 18:57:00,89.116667,hour
1,28038802,2185-12-20 09:59:00,108.800000,hour
2,21790335,2140-03-11 13:44:00,33.000000,hour
3,24357541,2177-02-10 14:00:00,659.416667,hour
4,23255460,2172-11-20 08:06:00,56.216667,hour
...,...,...,...,...
24674,29960248,2146-12-09 14:23:00,4408.000000,min
24675,29962016,2135-10-30 11:45:00,13085.000000,min
24676,29974575,2131-03-03 19:54:00,5934.000000,min
24677,29987115,2148-02-19 10:00:00,2640.000000,min
